In [2]:
import pandas as pd
import requests
import json
from config import *

In [3]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) # Format makes the output ans str variable.


In [4]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"
    params = {"query": city_name}
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': my_key}
    response = requests.get(url, headers = headers, params = params)
    return response.json()["Places"][0]["PlaceId"]

In [5]:
def flight_prices_jonty(outbound, inbound, date):
    x = city_code(outbound)
    y = city_code(inbound)
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{}/{}/{}".format(x, y, date)
    params = {"inboundpartialdate":date}

    #headers = {
    #    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #    'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': my_key}    

    response = requests.get(url, headers=headers, params=params)

    return response.json()

In [7]:
from  time import sleep

def flight_dates(outbound, inbound, start='yyyy-mm-dd', end='yyyy-mm-dd'):
    x = city_code(outbound)
    y = city_code(inbound)
    dates = get_dates(start, end)
    my_dict = {}
    for date in list(dates):
        try: 
            #print(date)
            #print("Gathering fligths")
            result = flight_prices_jonty(outbound, inbound, date)
            my_dict[date] = result
            sleep(5)
        except:
            print("Error from the API")
            sleep(60)
    return prettier(my_dict)

In [16]:
def prettier(dix):
#     flights = pd.DataFrame(columns=['date', 'price', 'flight_company'])
    prices = 1000000
    dates = ''
    flight_company = ''
    for key in dix.keys():
        price = int((dix[key]['Quotes'][0]['MinPrice']))
        if price < prices:
            prices = price
            dates= str(key)
            flight_company = str(dix[key]['Carriers'][0]['Name'])
    
    return print(f"cheapest flight:\nprice: {prices}, \ndate: {dates}, \nairline: {flight_company}")

In [17]:
flight_dates('London', 'Paris', '2021-06-15', '2021-06-16')

cheapest flight:
price: 41, 
date: 2021-06-16, 
airline: Vueling Airlines
